In [1]:
import numpy as np
import pandas as pd
from scipy.fftpack import fft, ifft, fftshift, ifftshift
from scipy.signal import hilbert
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
def VMD(signal, alpha=2000., tau=0., K=3, DC=0, init=1, tol=1e-7, N_iter=500):
    """
    Variational Mode Decomposition (VMD).
    signal: 1D array (real)
    alpha: balancing parameter of data-fidelity
    tau: time-step of the dual ascent (0 for noise-slack)
    K: number of modes
    DC: 0 -> no DC part, 1 -> allow a DC part
    init: 0 = all omegas = 0, 1 = uniformly spaced, 2 = random
    tol: convergence tolerance
    N_iter: maximum iterations
    Returns: modes (K, len(signal)), center freqs (K)
    """
    f = np.copy(signal).astype(float)
    N = len(f)
    # Mirror signal to reduce boundary effects (as in original code)
    T = N
    # time domain discrete samples
    t = np.arange(0, T) / T

    # Fourier domain discretization
    freqs = np.arange(0, T) / T
    # Fourier transform of signal
    f_hat = fft(f)
    f_hat = np.concatenate((f_hat[T//2:], f_hat[:T//2]))  # fftshift-like (centered)
    # initialize u_hat, omega
    u_hat = np.zeros((K, T), dtype=complex)
    omega = np.zeros(K)

    # initialize omegas
    if init == 1:
        omega = 0.5 * (np.arange(K) / K)  # uniformly spaced positive freqs
    elif init == 2:
        rng = np.random.default_rng(0)
        omega = rng.random(K)
    else:
        omega[:] = 0.0

    # if DC mode requested force first omega = 0
    if DC:
        omega[0] = 0.0

    # Lagrangian multipliers
    lambda_hat = np.zeros(T, dtype=complex)

    # main loop
    uDiff = tol + 1.0
    n = 0
    # Precompute frequency array (positive)
    k = np.concatenate((np.arange(0, T//2), np.arange(-T//2, 0)))
    freqs_full = k / T  # range -0.5 .. 0.5

    while (uDiff > tol) and (n < N_iter):
        u_hat_prev = u_hat.copy()
        # update each mode in frequency domain
        for i in range(K):
            # calculate residual excluding current mode
            sum_others = np.sum(u_hat, axis=0) - u_hat[i]
            # residual = f_hat - sum_others - lambda/2
            residual = f_hat - sum_others - lambda_hat / 2.0
            # update u_hat_i (equation for quadratic problem)
            denom = 1.0 + alpha * (freqs_full - omega[i])**2
            u_hat[i] = residual / denom

            # enforce DC condition
            if DC and i == 0:
                u_hat[i][freqs_full != 0] = 0.0

        # update omega (center frequencies)
        for i in range(K):
            # avoid division by zero: use real parts for numerator/denominator
            ui = u_hat[i]
            # numerator: sum(freq * |u_hat|^2)
            num = np.sum(freqs_full * (np.abs(ui)**2))
            den = np.sum(np.abs(ui)**2) + 1e-16
            omega[i] = num / den

        # dual ascent (update lambda)
        sum_u = np.sum(u_hat, axis=0)
        lambda_hat = lambda_hat + tau * (sum_u - f_hat)

        # compute convergence
        uDiff = np.sum([np.linalg.norm(u_hat[i] - u_hat_prev[i])**2 for i in range(K)])
        n += 1

    # reconstruct time-domain modes (unshifted)
    # inverse shift of u_hat to match ifft ordering
    # undo earlier fftshift-like concatenation:
    u_hat_unshift = np.concatenate((u_hat[:, T//2:], u_hat[:, :T//2]), axis=1)
    modes = np.real(ifft(u_hat_unshift, axis=1))
    return modes, omega

In [3]:
class RELM:
    def __init__(self, n_hidden=100, activation='tanh', C=1.0, random_state=None):
        self.n_hidden = int(n_hidden)
        self.activation = activation
        self.C = float(C)
        self.random_state = random_state
        self.is_fitted = False

    def _init_weights(self, n_features):
        rng = np.random.default_rng(self.random_state)
        self.W = rng.uniform(-1, 1, size=(self.n_hidden, n_features))
        self.b = rng.uniform(-1, 1, size=(self.n_hidden,))

    def _activation(self, X):
        if self.activation == 'sigmoid':
            X = np.clip(X, -500, 500)
            return 1.0 / (1.0 + np.exp(-X))
        if self.activation == 'tanh':
            return np.tanh(X)
        if self.activation == 'relu':
            return np.maximum(0.0, X)
        raise ValueError("Unknown activation")

    def fit(self, X, y):
        X = np.asarray(X); y = np.asarray(y)
        if y.ndim == 1:
            y = y.reshape(-1,1)
        N, d = X.shape
        self._init_weights(d)
        H = self._activation(X @ self.W.T + self.b)
        if N >= self.n_hidden:
            A = (np.eye(self.n_hidden) / self.C) + (H.T @ H)
            B = H.T @ y
            self.beta = np.linalg.solve(A, B)
        else:
            A = (np.eye(N) / self.C) + (H @ H.T)
            B = y
            self.beta = H.T @ np.linalg.solve(A, B)
        self.is_fitted = True
        return self

    def predict(self, X):
        if not self.is_fitted:
            raise RuntimeError("Model not fitted.")
        H = self._activation(np.asarray(X) @ self.W.T + self.b)
        Y = H @ self.beta
        return Y.ravel() if Y.shape[1] == 1 else Y


In [4]:
class GWO:
    def __init__(self, obj_func, lb, ub, dim, n_agents=12, n_iter=25, seed=42):
        self.obj = obj_func
        self.lb = np.array(lb, dtype=float)
        self.ub = np.array(ub, dtype=float)
        self.dim = dim
        self.n_agents = n_agents
        self.n_iter = n_iter
        self.rng = np.random.default_rng(seed)

    def optimize(self):
        wolves = self.rng.uniform(self.lb, self.ub, size=(self.n_agents, self.dim))
        fitness = np.array([self.obj(w) for w in wolves])
        idx = np.argsort(fitness)
        alpha, beta, delta = wolves[idx[0]].copy(), wolves[idx[1]].copy(), wolves[idx[2]].copy()
        f_alpha, f_beta, f_delta = float(fitness[idx[0]]), float(fitness[idx[1]]), float(fitness[idx[2]])
        for t in range(self.n_iter):
            a = 2 - 2 * (t / (self.n_iter - 1 + 1e-9))
            for i in range(self.n_agents):
                X = wolves[i].copy()
                for j in range(self.dim):
                    r1, r2 = self.rng.random(), self.rng.random()
                    A1 = 2 * a * r1 - a; C1 = 2 * r2
                    D_alpha = abs(C1 * alpha[j] - X[j]); X1 = alpha[j] - A1 * D_alpha

                    r1, r2 = self.rng.random(), self.rng.random()
                    A2 = 2 * a * r1 - a; C2 = 2 * r2
                    D_beta = abs(C2 * beta[j] - X[j]); X2 = beta[j] - A2 * D_beta

                    r1, r2 = self.rng.random(), self.rng.random()
                    A3 = 2 * a * r1 - a; C3 = 2 * r2
                    D_delta = abs(C3 * delta[j] - X[j]); X3 = delta[j] - A3 * D_delta

                    X[j] = (X1 + X2 + X3) / 3.0
                wolves[i] = np.clip(X, self.lb, self.ub)
            fitness = np.array([self.obj(w) for w in wolves])
            idx = np.argsort(fitness)
            if fitness[idx[0]] < f_alpha:
                alpha, f_alpha = wolves[idx[0]].copy(), float(fitness[idx[0]])
            if fitness[idx[1]] < f_beta:
                beta, f_beta = wolves[idx[1]].copy(), float(fitness[idx[1]])
            if fitness[idx[2]] < f_delta:
                delta, f_delta = wolves[idx[2]].copy(), float(fitness[idx[2]])
        return alpha, f_alpha

In [5]:
def create_multivariate_lagged_dataset(df, target_col, feature_cols, lag=3):
    data = df[feature_cols].values
    target_idx = feature_cols.index(target_col)
    X, y = [], []
    for i in range(lag, len(df)):
        X.append(data[i-lag:i].flatten())
        y.append(data[i, target_idx])
    return np.array(X), np.array(y)

def safe_mape(y_true, y_pred, min_denom=1.0):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    mask = np.abs(y_true) >= min_denom
    if np.sum(mask) == 0:
        return np.nan
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100)

def sde(y_true, y_pred):
    return float(np.std(np.asarray(y_true) - np.asarray(y_pred)))


In [6]:
def decode_relm_params(position, Hmin=20, Hmax=500, Cmin_log=-4, Cmax_log=4):
    h_raw, logC, act_raw = position
    n_hidden = int(np.round(Hmin + np.clip(h_raw, 0, 1) * (Hmax - Hmin)))
    n_hidden = int(np.clip(n_hidden, Hmin, Hmax))
    C = 10.0 ** float(np.clip(logC, Cmin_log, Cmax_log))
    act_idx = int(np.round(np.clip(act_raw, 0, 2)))
    activation = ['tanh', 'sigmoid', 'relu'][act_idx]
    return n_hidden, C, activation

def make_relm_objective(X_train, y_train, X_val, y_val, random_state=42,
                        Hmin=20, Hmax=500, Cmin_log=-4, Cmax_log=4):
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)
    def objective(position):
        n_hidden, C, activation = decode_relm_params(position, Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)
        try:
            model = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            model.fit(X_train_s, y_train)
            y_pred = model.predict(X_val_s)
            return float(np.sqrt(mean_squared_error(y_val, y_pred)))
        except Exception:
            return 1e6
    return objective

In [7]:
def vmd_gwo_relm_pipeline(
    df,
    target_column,
    feature_columns,
    lag_steps=12,
    K_modes=4,
    vmd_alpha=2000.,
    vmd_tau=0.,
    vmd_init=1,
    vmd_DC=0,
    vmd_tol=1e-7,
    vmd_Niter=500,
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=500,
    Cmin_log=-4, Cmax_log=4,
    max_step_eval=7
):
    """
    Returns dict with:
      - per_mode_info (list of best params)
      - one_step_metrics (on reconstructed forecast)
      - multistep_df (direct multi-step reconstructed metrics)
      - modes (array K x L), reconstructed prediction array, and true test array
    """
    # 1) VMD decompose target
    signal = df[target_column].values.astype(float)
    modes, omegas = VMD(signal, alpha=vmd_alpha, tau=vmd_tau, K=K_modes, DC=vmd_DC, init=vmd_init, tol=vmd_tol, N_iter=vmd_Niter)
    # modes shape (K, L)

    L = modes.shape[1]
    # Build identical lagging scheme as earlier: lag removes first `lag_steps` samples
    n_samples = L - lag_steps
    if n_samples <= 0:
        raise ValueError("lag_steps too large for series length after VMD.")
    train_end = int(0.7 * n_samples)
    val_end = int(0.85 * n_samples)

    per_mode_info = []
    test_mode_preds = []
    test_mode_lengths = []

    for m in range(K_modes):
        mode_series = modes[m, :]
        df_mode = df.copy()
        df_mode[target_column] = mode_series

        X_mode, y_mode = create_multivariate_lagged_dataset(df_mode, target_column, feature_columns, lag=lag_steps)
        X_train, y_train = X_mode[:train_end], y_mode[:train_end]
        X_val, y_val     = X_mode[train_end:val_end], y_mode[train_end:val_end]
        X_test, y_test   = X_mode[val_end:], y_mode[val_end:]

        if len(X_train) < 5 or len(X_val) < 1 or len(X_test) < 1:
            print(f"[Mode {m+1}] insufficient data for train/val/test splits; skipping.")
            per_mode_info.append({"mode": m+1, "skipped": True})
            test_mode_preds.append(np.zeros_like(X_test[:,0]) if X_test.size else np.array([]))
            test_mode_lengths.append(len(X_test))
            continue

        # objective and GWO
        obj = make_relm_objective(X_train, y_train, X_val, y_val, random_state=random_state,
                                  Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)

        lb = np.array([0.0, Cmin_log, 0.0], dtype=float)   # [h_raw, logC, act_raw]
        ub = np.array([1.0, Cmax_log, 2.0], dtype=float)

        gwo = GWO(obj, lb, ub, dim=3, n_agents=gwo_agents, n_iter=gwo_iters, seed=random_state + m)
        best_pos, best_fit = gwo.optimize()
        n_hidden, C, activation = decode_relm_params(best_pos, Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)

        # train final on train+val and predict test
        scaler = StandardScaler()
        X_trval = np.vstack([X_train, X_val])
        y_trval = np.concatenate([y_train, y_val])
        X_trval_s = scaler.fit_transform(X_trval)
        X_test_s = scaler.transform(X_test)

        model = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
        model.fit(X_trval_s, y_trval)
        y_pred_test_mode = model.predict(X_test_s)

        per_mode_info.append({
            "mode": m+1,
            "best_val_rmse": float(best_fit),
            "n_hidden": int(n_hidden),
            "C": float(C),
            "activation": activation,
            "test_len": int(len(y_test))
        })

        test_mode_preds.append(y_pred_test_mode)
        test_mode_lengths.append(len(y_test))

    # Align test predictions and reconstruct by summation
    n_test = None
    for ln in test_mode_lengths:
        if ln:
            n_test = ln
            break
    if n_test is None:
        raise RuntimeError("No valid test length across modes.")

    stacked = []
    for arr in test_mode_preds:
        a = np.asarray(arr)
        if len(a) == n_test:
            stacked.append(a)
        elif len(a) == 0:
            stacked.append(np.zeros(n_test))
        elif len(a) < n_test:
            stacked.append(np.concatenate([a, np.zeros(n_test - len(a))]))
        else:
            stacked.append(a[:n_test])
    stacked = np.array(stacked)  # (K, n_test)

    # reconstructed forecast
    y_pred_reconstructed = np.sum(stacked, axis=0)
    # true test (original df target aligned)
    y_true_test = df[target_column].values[lag_steps + val_end : lag_steps + val_end + n_test]

    # One-step metrics
    one_mae = mean_absolute_error(y_true_test, y_pred_reconstructed)
    one_rmse = np.sqrt(mean_squared_error(y_true_test, y_pred_reconstructed))
    one_mape = safe_mape(y_true_test, y_pred_reconstructed)
    one_sde = sde(y_true_test, y_pred_reconstructed)

    one_step_metrics = {"MAE": float(one_mae), "RMSE": float(one_rmse), "MAPE (%)": float(one_mape) if not np.isnan(one_mape) else np.nan, "SDE": float(one_sde)}

    # Multi-step direct predictions: for each step, retrain each mode's model on TRAIN only with tuned params and predict X_test[:-step]
    multistep_rows = []
    for step in range(1, max_step_eval + 1):
        mode_step_preds = []
        valid = True
        for info in per_mode_info:
            if info.get("skipped", False):
                mode_step_preds.append(np.zeros(max(0, n_test - step)))
                continue
            midx = info["mode"] - 1
            # rebuild mode dataset
            mode_series = modes[midx, :]
            df_mode = df.copy(); df_mode[target_column] = mode_series
            X_mode, y_mode = create_multivariate_lagged_dataset(df_mode, target_column, feature_columns, lag=lag_steps)
            X_train, y_train = X_mode[:train_end], y_mode[:train_end]
            X_test_all, y_test_all = X_mode[val_end:], y_mode[val_end:]
            if X_test_all.shape[0] <= step:
                valid = False; break
            X_test_step = X_test_all[:-step]
            # fit on TRAIN only with tuned params
            n_hidden = info["n_hidden"]
            C = info["C"]
            activation = info["activation"]
            scaler_train = StandardScaler()
            X_train_s = scaler_train.fit_transform(X_train)
            X_test_step_s = scaler_train.transform(X_test_step)
            model_step = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            model_step.fit(X_train_s, y_train)
            y_pred_step_mode = model_step.predict(X_test_step_s)
            mode_step_preds.append(y_pred_step_mode)
        if not valid:
            break
        mode_step_preds = np.array(mode_step_preds)  # (K, n_test-step)
        y_pred_step_recon = np.sum(mode_step_preds, axis=0)
        y_true_step = df[target_column].values[lag_steps + val_end + step : lag_steps + val_end + step + y_pred_step_recon.shape[0]]
        L_true = len(y_true_step); L_pred = len(y_pred_step_recon)
        mlen = min(L_true, L_pred)
        if mlen == 0:
            break
        y_true_step = y_true_step[:mlen]; y_pred_step_recon = y_pred_step_recon[:mlen]
        multistep_rows.append({
            "Step": step,
            "MAE": float(mean_absolute_error(y_true_step, y_pred_step_recon)),
            "RMSE": float(np.sqrt(mean_squared_error(y_true_step, y_pred_step_recon))),
            "MAPE (%)": float(safe_mape(y_true_step, y_pred_step_recon)),
            "SDE": float(sde(y_true_step, y_pred_step_recon))
        })

    multistep_df = pd.DataFrame(multistep_rows)

    return {
        "per_mode_info": per_mode_info,
        "one_step_metrics": one_step_metrics,
        "multistep_df": multistep_df,
        "modes": modes,
        "omegas": omegas,
        "y_true_test": y_true_test,
        "y_pred_reconstructed": y_pred_reconstructed
    }


In [8]:
feature_columns = ['AirTemp','Azimuth','CloudOpacity','DewpointTemp','Dhi','Dni','Ebh',
                   'WindDirection10m','Ghi','RelativeHumidity','SurfacePressure','WindSpeed10m']
df = pd.read_csv('/Users/hrishityelchuri/Documents/windPred/raw/8.52 hrishit data.csv')

df['PeriodEnd'] = pd.to_datetime(df['PeriodEnd'])
df['PeriodStart'] = pd.to_datetime(df['PeriodStart'])
df = df.sort_values('PeriodEnd')

In [9]:
res = vmd_gwo_relm_pipeline(
    df,
    target_column='WindSpeed10m',
    feature_columns=feature_columns,
    lag_steps=12,
    K_modes=4,
    vmd_alpha=2000.,
    vmd_tau=0.,
    vmd_init=1,
    vmd_DC=0,
    vmd_tol=1e-7,
    vmd_Niter=500,
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=400,
    Cmin_log=-4, Cmax_log=4,
    max_step_eval=7
)

print("Per-mode best params:")
print(pd.DataFrame(res['per_mode_info']))
print("\nOne-step reconstructed metrics:")
print(res['one_step_metrics'])
print("\nMulti-step reconstructed metrics:")
print(res['multistep_df'].to_string(index=False))

Per-mode best params:
   mode  best_val_rmse  n_hidden             C activation  test_len
0     1       0.167591       389  10000.000000    sigmoid     20612
1     2       0.049934       396      0.215079       relu     20612
2     3       0.049497       396      0.000468       relu     20612
3     4       0.082513       399      0.000254       relu     20612

One-step reconstructed metrics:
{'MAE': 0.3713936275258358, 'RMSE': 0.47939256219583354, 'MAPE (%)': 14.015372891320958, 'SDE': 0.47938999193522863}

Multi-step reconstructed metrics:
 Step      MAE     RMSE  MAPE (%)      SDE
    1 0.445964 0.582369 16.944894 0.578042
    2 0.540181 0.695374 20.165961 0.691748
    3 0.650721 0.824224 24.084771 0.821160
    4 0.760760 0.953001 27.951795 0.950345
    5 0.864433 1.075115 31.607420 1.072754
    6 0.960249 1.186708 35.054251 1.184562
    7 1.046611 1.286073 38.162725 1.284084
